In [2]:
import pandas as pd

In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [2]:
#import cohere

In [3]:
# Paste your API key here. Remember to not share publicly
#api_key = 'qPKnuy0sl0o4eg2bQ9nOe6qVmzTZoYcZT28DXvoN'

# Create and retrieve a Cohere API key from os.cohere.ai
#co = cohere.Client(api_key)

In [3]:
import dvc.api as DvcApi

In [9]:
path = "/data/Example_data1.csv"
repo = "https://github.com/emtinanseo/Prompt-Engineering.git"
version = "v1"
remote = "lremote"

content = DvcApi.get_url(path = path, repo = repo, rev = version, remote = remote) #could be tag or git commit

RemoteNotFoundError: config file error: remote 'lremote' doesn't exist